<a href="https://colab.research.google.com/github/isDaisyorDaisy/proejcts/blob/main/Conversion_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


The A/B testing method determines which of two versions of something produces the best results.
https://www.oracle.com/cx/marketing/what-is-ab-testing/ 
why to use a/b testing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('sample ab_data.csv')
df

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,1
3,853541,28:03.1,treatment,new_page,1
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294473,751197,28:38.6,control,old_page,0
294474,945152,51:57.1,control,old_page,0
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0


missing value


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


messed data and orgnized data

In [ ]:
#((df.group== 'treatment')&(df.landing_page=='new_page')).sum()##return true or false
((df['group'] =='treatment')& (df['landing_page']=='new_page')).sum()


145311

In [ ]:
((df.group == 'control')&(df.landing_page =='old_page')).sum()##return true or false

145274

In [ ]:
df.correct_data = (((df.group== 'treatment')&(df.landing_page=='new_page')) | ((df.group == 'control')&(df.landing_page =='old_page')))
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,1
3,853541,28:03.1,treatment,new_page,1
4,864975,52:26.2,control,old_page,1


In [ ]:
df = df.drop(df[df.correct_data == False].index)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290585 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


user info


In [ ]:
df.user_id.nunique()

290584

value_counts() 
sort_values()

In [ ]:
df.user_id.value_counts().sort_values(ascending = False).head()

773192    2
834487    1
936923    1
679687    1
719014    1
Name: user_id, dtype: int64

In [ ]:
df[df.user_id == 773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,37:58.8,treatment,new_page,1
2893,773192,55:59.6,treatment,new_page,0


In [ ]:
df.drop(2893, axis = 0, inplace=True)

In [ ]:
df['user_id'].value_counts().sort_values(ascending = False).head()

851104    1
834487    1
936923    1
679687    1
719014    1
Name: user_id, dtype: int64

In [ ]:
df[['user_id', 'group']].groupby('group').count()

,user_id
group,
control,145274
treatment,145310


In [ ]:
#df[['user_id', 'group', 'converted']].groupby('group').agg({'user_id':'count', 'converted': 'mean', 'converted':'sum'})???????
df[['user_id','group','converted']].groupby('group').agg({'user_id':'count','converted':'mean'})

,user_id,converted
group,,
control,145274,0.120386
treatment,145310,0.125360


In [ ]:
convert_old =  df[df['group']=='control'].converted.sum()
convert_old

17489

In [ ]:
convert_new = df[df['group']=='treatment'].converted.sum()
convert_new

18216

In [ ]:
old = len(df[df['group']=='control'].converted)
#count value_counts
old

145274

In [ ]:
new = len(df[df['group']=='treatment'].converted)
type(new)

int

In [ ]:
conversion_dic = {'views':{'control': old, 'treatment':new}, 'converts':{'control':convert_old, 'treatment':convert_new}}
##how to get a new df from the old df
conversion_table = pd.DataFrame(conversion_dic)
#conversion_table
conversion_table['conversion%'] = conversion_table['converts'] / conversion_table['views']
conversion_table['conversion%']= conversion_table['conversion%'].apply(lambda x:str(round(x,3)*100) +'%')#np.min np.round
conversion_table


,views,converts,conversion%
control,145274,17489,12.0%
treatment,145310,18216,12.5%


Z-test

**null:** **$p_{new}$** - **$p_{old}$** <=0

**alternative:** **$p_{new}$** - **$p_{old}$** >0




In [ ]:
def z_test(p1, p0, n1, n0):
  delta = p1-p0
  p = (p1*n1+p0*n0)/(n1+n0)
  return delta/np.sqrt(p*(1-p)*(1/n1+1/n0))

  # proportion

In [ ]:
p0 = convert_old/old
p1 = convert_new/new
n1 = new
n0 = old
p1,p0,n1,n0

(0.12535957607872825, 0.1203863045004612, 145310, 145274)

In [ ]:
z_value = z_test(p1,p0,n1,n0)
z_value

4.083082708291715

In [ ]:
from scipy.stats import norm

In [ ]:
p_value = 1- norm.cdf(z_value)##p_value
p_value

2.2221098369135106e-05

p_value < 0.05 reject null

In [ ]:
import statsmodels.api as sm

p1>p0. stat, pval = proportions_ztest(count, nobs, value) 
p_value compare with 0.05

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_new, convert_old], [new, old], alternative = 'larger')
z_score, p_value

(4.083082708291715, 2.2221098369156668e-05)